# Hypersparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperparameter-exp'

experiment=Experiment(ws, experiment_name)

In [3]:
cpu_cluster_name = "cpu-cluster2"

try: #--- Verify that cluster does not exist already
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [4]:
key = "heart-dataset"
description_text = "Heart Disease dataset"

#-- Dataset Call
if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
df = dataset.to_pandas_dataframe()
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [5]:
#-- Preprocessing Data
## Input is X, Output is Y
## Performing Standardization on Bloodpressure, Cholestrol, Thalac
## Binning Age grp into 7 groups
##Drop null rows

### standardization
x_df = dataset.to_pandas_dataframe().dropna() #drop any rows with null values
y_df = x_df.pop("target")
x_df[['trestbps', 'chol', 'thalach']] = StandardScaler().fit_transform(x_df[['trestbps', 'chol', 'thalach']])
#print(x_df)
#print(y_df)

### binning
min_value = df['age'].min()
max_value = df['age'].max()
bins = np.linspace(min_value,max_value,7)
#print(bins)
labels = ['29-37', '38-45', '46-53', '54-61', '62-69', '70-77']
x_df['age_bins'] = pd.cut(x_df['age'], bins=bins, labels=labels, include_lowest=True)
drop_age = x_df.pop("age") #drop age column

### rearrange columns
x_df = x_df[['age_bins', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
#pd.set_option('display.max_rows', x_df.shape[0]+1)
#print(x_df)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [21]:
from azureml.train.hyperdrive.sampling import GridParameterSampling
from azureml.train.hyperdrive.policy import MedianStoppingPolicy
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = GridParameterSampling( {
        "max_iter": choice(range(10,31)) ,
        #"batch_size": choice(16,32),
    }
)

# #TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory="./",
                    compute_target=cpu_cluster,
                    entry_script='dataprocess.py'
                   )

hyperdrive_run_config =  HyperDriveConfig(estimator=estimator,
                      hyperparameter_sampling=param_sampling,
                      policy= early_termination_policy,
                      primary_metric_name="Accuracy",
                  primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                      max_total_runs=20,
                      max_concurrent_runs=4)

In [22]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_a6e0455f-cd60-4041-a03f-58d421b4b019
Web View: https://ml.azure.com/runs/HD_a6e0455f-cd60-4041-a03f-58d421b4b019?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-142523/workspaces/quick-starts-ws-142523&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-12T20:49:18.265624][API][INFO]Experiment created<END>\n""<START>[2021-04-12T20:49:18.788597][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-04-12T20:49:19.017677][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-04-12T20:49:19.7725424Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_a6e0455f-cd60-4041-a03f-58d421b4b019
Web View: https://ml.azure.com/runs/HD_a6e0455f-cd60-4041-a03f-58d421b4b019?wsid=/subscriptions/9a7511b8-150f-4a58-8

AssertionError: 

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service